Training notebook:
https://www.kaggle.com/mostafaibrahim17/yolov5-vinbigdata

Credits:
https://www.kaggle.com/awsaf49/vinbigdata-2-class-filter

https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer

https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import shutil

In [2]:
# load yolo submission
yolo = pd.read_csv('../input/yolo169/submission_2.csv')
effnetb6 = pd.read_csv('../input/vinbigdata-2class-prediction/2-cls test pred.csv') # AUC:0.98

In [3]:
yolo.head()

,image_id,PredictionString
0,83caa8a85e03606cf57e49147d7ac569,3 0.7 792 1181 1760 1564
1,7550347fa2bb96c2354a3716dfa3a69c,5 0.7 1735 1161 2226 2134
2,74b23792db329cff5843e36efb8aa65a,14 1 0 0 1 1
3,94568a546be103177cb582d3e91cd2d8,0 0.7 989 1025 1215 1334 3 0.8 643 1628 1503 1932
4,6da36354fc904b63bc03eb3884e0c35c,13 0.2 1510 1132 1618 1281 11 0.2 1305 297 150...


In [4]:
effnetb6.head()

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [5]:
pred = pd.merge(yolo, effnetb6, on = 'image_id', how = 'left')
pred.head()

,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,3 0.7 792 1181 1760 1564,0.970583
1,7550347fa2bb96c2354a3716dfa3a69c,5 0.7 1735 1161 2226 2134,0.039873
2,74b23792db329cff5843e36efb8aa65a,14 1 0 0 1 1,0.010240
3,94568a546be103177cb582d3e91cd2d8,0 0.7 989 1025 1215 1334 3 0.8 643 1628 1503 1932,0.065679
4,6da36354fc904b63bc03eb3884e0c35c,13 0.2 1510 1132 1618 1281 11 0.2 1305 297 150...,0.838772


In [6]:
low_thr  = 0.08
high_thr = 0.95

In [7]:
def filter_2cls(row, low_thr=low_thr, high_thr=high_thr):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [8]:
sub = pred.apply(filter_2cls, axis=1)
sub.head()

,image_id,PredictionString,target
0,83caa8a85e03606cf57e49147d7ac569,3 0.7 792 1181 1760 1564,0.970583
1,7550347fa2bb96c2354a3716dfa3a69c,14 1 0 0 1 1,0.039873
2,74b23792db329cff5843e36efb8aa65a,14 1 0 0 1 1,0.010240
3,94568a546be103177cb582d3e91cd2d8,14 1 0 0 1 1,0.065679
4,6da36354fc904b63bc03eb3884e0c35c,13 0.2 1510 1132 1618 1281 11 0.2 1305 297 150...,0.838772


In [9]:
sub[['image_id', 'PredictionString']].to_csv('submission.csv',index = False)